In [28]:
import sympy as sym
import numpy as np
import scipy.linalg as sla

# Sistemas dados en el primer punto

In [29]:
A2 = np.array([[1, -8, -2], [1, 1, 5], [3, -1, 1]])
b2 = np.array([[1], [4], [-2]])

A3 = np.array([[1, 4, 0], [0, 1, 1], [2, 0, 3]])
b3 = np.array([[5], [2], [0]])

A4 = np.array([[1, 3, -1], [4, -1, 1], [1, 1, 7]])
b4 = np.array([[18], [27.34], [16.2]])

# Sistema del punto 2

In [30]:
A1 = np.array([[4, 3, 0], [3, 4, -1], [0, -1, 4]])
b1 = np.array([[0.254], [-1.425], [2.978]])

# Aclaración de porque los sistemas del punto uno no se les puede aplicar cholesky

## Primero imprimimos los valores propios del primer sistema del punto 1
### Como ejemplo usaremos el primer sistema

In [31]:
np.linalg.eigvalsh(A2)

array([-2.56155281,  1.56155281,  4.        ])

## Para poder realizar cholesky con la matriz dada, esa matriz A require ser definidio positivo, y para esto todos los valores propios de $  A + A^{T}  $ son positivos.

## Teniendo en cuenta lo anterior se realizó una pequeña función que nos retorne verdadero en el caso que cumpla con la condición descrita anteriormente, y falso en el caso contrario.

In [32]:
def verificacion(A):
    return np.all(np.linalg.eigvals(A+A.transpose()) > 0) # Verificar que A sea definida positiva

In [33]:
print("La matriz A2 es definida positiva:", verificacion(A2))
print("La matriz A3 es definida positiva:", verificacion(A3))
print("La matriz A4 es definida positiva:", verificacion(A4))

La matriz A2 es definida positiva: False
La matriz A3 es definida positiva: False
La matriz A4 es definida positiva: False


## Como podemos ver los tres sistemas dados en el punto 1 no cumplen la condición de positivo definido.

## Ahora probaremos con la matriz A dada en el punto 2.

In [34]:

print("La matriz A1 es definida positiva:", verificacion(A1))

La matriz A1 es definida positiva: True


## La matriz A dada en el segundo punto si cumple con la condición necesaria por lo cual para proseguir con la explicación de cholesky usaremos esta matriz

# Método Chloesky y solución del sistema implementación propia

In [35]:
def forward_substitution(L, b):

    n = L.shape[0]

    y = np.zeros_like(b, dtype=np.double);
    
    y[0] = b[0] / L[0, 0]

    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

In [36]:
def cholesky(A):
    
    n = A.shape[0]
    
    L = np.zeros((n, n), dtype=np.double)
    
    for k in range(n):
        
        L[k, k] = np.sqrt(A[k, k] - np.sum(L[k, :] ** 2))
        
        L[(k+1):, k] = (A[(k+1):, k] - L[(k+1):, :] @ L[:, k]) / L[k, k]
    
    return L

In [37]:
L = cholesky(A1)
sym.Matrix(L)

Matrix([
[2.0,                0.0,             0.0],
[1.5,    1.3228756555323,             0.0],
[0.0, -0.755928946018454, 1.8516401995451]])

## Más abajo en el proceso realizado con librería numpy y scipy se va a realiz nuevamente cholesky con estas librerias, y al comparar las descomposiciones se observa que dan el mismo resultado.

## Antes de proseguir a esa sección en la cuál también se procederá a dar solución al sistema daremos respuesta a las operaciones necesarias para aplicar el algoritmo.

## Analizando varias implementaciónes de Cholesky junto con esta podemos ver que el algoritmo tiene de complejidad.

# $$ O(n^{3}/3)  $$

## Y si queremos agregar la resolución de los sistemas triangulares se le sumaría $$ 2n^{2}  $$

# Método Cholesky y solución del sistema con librería numpy y scipy

## Probando función cholesky

In [38]:
A = np.array([[1,-2j],[2j,5]])
sym.Matrix(A)

Matrix([
[  1.0, -2.0*I],
[2.0*I,    5.0]])

In [39]:
L = np.linalg.cholesky(A)
sym.Matrix(L)

Matrix([
[  1.0,   0],
[2.0*I, 1.0]])

In [40]:
res = np.dot(L, L.T.conj()) # Verificar que L * L.H = A
sym.Matrix(res)

Matrix([
[  1.0, -2.0*I],
[2.0*I,    5.0]])

## Uso cholesky solución sistema pedido en el ejercicio del Taller

In [41]:
#A = np.array([[36, 30, 18], [30,41,23], [18, 23, 14]])
#b = np.array([[288], [296], [173]])
A = A1
sym.Matrix(A)


Matrix([
[4,  3,  0],
[3,  4, -1],
[0, -1,  4]])

In [42]:
b = b1
sym.Matrix(b)

Matrix([
[ 0.254],
[-1.425],
[ 2.978]])

### Cholesky.

In [43]:
#L = sla.cholesky(A, lower=True)
L = np.linalg.cholesky(A)
sym.Matrix(L)

Matrix([
[2.0,                0.0,             0.0],
[1.5,    1.3228756555323,             0.0],
[0.0, -0.755928946018454, 1.8516401995451]])

### Comprobar si la descomposición se hizo bien.

In [44]:
LT = L.T.conj()  # matriz traspuesta
sym.Matrix(LT)

# L*LT = A
A_cholesky = np.dot(L, LT)  # debe dar la matriz original A
sym.Matrix(A_cholesky)

Matrix([
[4.0,  3.0,  0.0],
[3.0,  4.0, -1.0],
[0.0, -1.0,  4.0]])

### Como podemos ver el A_cholesky es el mismo que A por lo que la descomposición fue correcta

### Ahora procedemos a solucionar el sistema con la descomposición obtenida de cholesky

In [45]:
# Solución del sistema con el resultado de la decomposición
# L Y = B
y = forward_substitution(L, b)

# x = LT*y (solución del sistema)
x = np.linalg.solve(LT, y)
sym.Matrix(x)

Matrix([
[             0.499],
[-0.580666666666667],
[ 0.599333333333333]])

### Usamos otra función para resolver la matriz y si el resultado obtenido es igual al dado anteriormente significa que la solución del sistema fue correcta.

In [46]:
check_x = sla.solve(A, b)
sym.Matrix(check_x)

Matrix([
[             0.499],
[-0.580666666666667],
[ 0.599333333333333]])